# How to use multiple Neptune integrations together

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/how-to-guides/multiple-integrations/notebooks/Multiple_integrations.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

<a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/how-to-guides/multiple-integrations">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a>

<a target="_blank" href="https://app.neptune.ai/o/showcase/org/Multiple-integrations/runs/details?viewId=standard-view&detailsTab=metadata&shortId=MUL-1"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a>

<a target="_blank" href="https://docs.neptune.ai/tutorials/multiple-integrations/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

Neptune's integrations work independently of each other. This means that you can use multiple integrations together.

For example, if you are doing hyperparameter optimization using Optuna on scikit-learn models, you can use both our Optuna and scikit-learn integrations together in the same script. The [Optuna integration](https://docs.neptune.ai/integrations/optuna/) will log all the relevant Optuna Study and Trial metadata, while the [scikit-learn integration](https://docs.neptune.ai/integrations/sklearn/) will log the scikit-learn model metadata. If you use Airflow to schedule the above setup, you can add our [Airflow integration](https://docs.neptune.ai/integrations/airflow/) to automatically log Airflow metadata.

However, since the integrations work independently, it is necessary to organize the logged metadata in a way that makes it easy for you to search for the information you need, without it being overwritten by another integration.

This notebook shows how you can use Neptune's Optuna and scikit-learn integrations together. For an example of a similar setup with Airflow scheduling on top, check this script &rarr; [Multiple_integrations.py](../scripts/Multiple_integrations.py).

The concepts remain the same irrespective of the integration you are using. Check the list of all Neptune's integrations here &rarr; [Integrations](https://docs.neptune.ai/integrations/)

## Before you start

This notebook example lets you try out Neptune anonymously, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -q -U neptune[sklearn,optuna] scikit-learn optuna

## Prepare the dataset

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

data, target = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25)

## Create the Optuna objective function

We'll use the scikit-learn integration inside the objective function to log model metadata for all trained models in each trial.

**💡 Tip**  
If you want to compare metadata across different trials, you can create a separate run for each trial as described here &rarr; [Logging each trial as separate Neptune run](https://docs.neptune.ai/integrations/optuna/#logging-each-trial-as-separate-neptune-run)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from neptune.integrations.sklearn import create_regressor_summary


def objective(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 2, 64),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "min_samples_split": trial.suggest_int("min_samples_split", 3, 10),
    }

    # Train the model
    model = RandomForestRegressor(**param)
    model.fit(X_train, y_train)

    # Log model summary for each trial under the "sklearn" namespace
    run[f"sklearn/model_summary_{trial.number}"] = create_regressor_summary(
        model, X_train, X_test, y_train, y_test
    )

    # Fetch objective score from the run
    run.wait()
    score = run[f"sklearn/model_summary_{trial.number}/test/scores/mean_absolute_error"].fetch()

    return score

## Create the Optuna study and Neptune run

In [ ]:
import optuna

study = optuna.create_study(direction="minimize")

In [ ]:
import neptune

run = neptune.init_run(
    project="common/multiple-integrations",  # Replace with your own
    api_token=neptune.ANONYMOUS_API_TOKEN,  # Replace with your own
    capture_hardware_metrics=True,
    capture_stderr=True,
    capture_stdout=True,
    tags=["notebook", "optuna", "sklearn"],
    dependencies="infer",
)

## Initialize Neptune's Optuna callback
The below will log the Optuna sweeps and trials to the study-level run under the "optuna" namespace.

In [ ]:
from neptune.integrations.optuna import NeptuneCallback

neptune_optuna_callback = NeptuneCallback(run, base_namespace="optuna")

## Run the hyperparameter-sweep with Neptune's Optuna callback

In [ ]:
study.optimize(objective, n_trials=3, callbacks=[neptune_optuna_callback])

## Stop the run

In [ ]:
run.stop()

## Explore the results in Neptune
Follow the link to open the run in Neptune. You can explore the scikit-learn and Optuna metadata in the "sklearn" and "optuna" namespaces, respectively.

You can also check out this [example run](https://app.neptune.ai/o/showcase/org/multiple-integrations/runs/details?viewId=standard-view&detailsTab=metadata&shortId=MUL-2).